In [78]:
import numpy as np
from sklearn import datasets

In [79]:
data = datasets.load_iris().data
target = datasets.load_iris().target
c = np.column_stack([data, target])

df = pd.DataFrame(c, columns=['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target'])
df1 = df.query("target > 0")
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 50 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  100 non-null    float64
 1   sepal width (cm)   100 non-null    float64
 2   petal length (cm)  100 non-null    float64
 3   petal width (cm)   100 non-null    float64
 4   target             100 non-null    float64
dtypes: float64(5)
memory usage: 4.7 KB


In [80]:
X = np.delete(df1.values, 4, 1)
X[:5]

array([[7. , 3.2, 4.7, 1.4],
       [6.4, 3.2, 4.5, 1.5],
       [6.9, 3.1, 4.9, 1.5],
       [5.5, 2.3, 4. , 1.3],
       [6.5, 2.8, 4.6, 1.5]])

In [81]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df1['target'])
Y = pd.Series(data = le.transform(df1['target']))
Y = Y.to_numpy()
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [82]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, train_size=0.3, random_state=63, stratify=Y)

from sklearn.preprocessing import StandardScaler
SC = StandardScaler()
x_t = SC.fit_transform(Xtrain)
x_t[:5]

array([[-0.74644375, -0.37828438, -0.96220013, -1.00089226],
       [-1.05963694, -1.83322428, -1.08662256, -1.00089226],
       [ 1.44590853,  0.20369159,  1.27740362,  0.79175059],
       [-0.74644375, -0.08729639, -0.8377777 , -1.00089226],
       [-1.21623353,  0.20369159, -0.46451041, -0.55273155]])

In [83]:
def sigmoid(a):
    return 1.0 / (1 + np.exp(-a))

def cost(x, y, theta, m):
    h = sigmoid(np.matmul(x, theta))
    cost2 = (-np.log(h[y == 1]).sum()      -  np.log(1 - h[y == 0]).sum()) /m
    #cost = (np.matmul(-y.T, np.log(h))  -  np.matmul((1 - y.T), np.log(1 - h))) /m
    return cost2

def gradient(theta, x, y, m):
    h = sigmoid(np.matmul(x, theta))
    grad = np.matmul(x.T, (h - y)) / m
    return grad

def gradient_Descent(theta, alpha, grad):
    theta = theta - alpha * grad
    return theta

In [84]:
Theta = np.array([0, 0, 0, 0, 0])
#ones = np.ones(X.shape[0])
ones = np.array([[1] * x_t.shape[0]]).T
Xd = np.hstack([ones, x_t])

m = x_t.shape[0]

## Градиентный спуск:

In [85]:
# Градиентный спуск
Theta = np.array([0, 0, 0, 0, 0])
cost_prev = np.inf
learning_rate = 0.1
tol = 1e-7
max_iter = 1e7

for i in range(int(max_iter)):    
    grad = gradient(Theta, Xd, ytrain, m)
    Theta = Theta - learning_rate * grad
    loss = cost(Xd, ytrain, Theta, m)
    if cost_prev - loss < tol:
        break
    cost_prev = loss

print('Количество шагов:', i)
print('cost:', cost(Xd, ytrain, Theta, m), 'Theta:', Theta)

Xtest_scaled = SC.transform(Xtest)
Xtest_scaled = np.c_[np.ones(Xtest_scaled.shape[0]), Xtest_scaled]
y_pred = sigmoid(np.matmul(Xtest_scaled, Theta))
print("Доля правильно предсказанных классов:", np.sum(ytest == np.where(y_pred >= 0.5, 1, 0))/len(ytest))

Количество шагов: 37128
cost: 0.004136405426724645 Theta: [ 1.19071738  0.69232861  1.2452645   4.28365669 14.06206126]
Доля правильно предсказанных классов: 0.9142857142857143


## Градиентный спуск методом Нестерова:

In [86]:
# Градиентный спуск методом Нестерова

Theta = np.array([0, 0, 0, 0, 0])
cost_prev = np.inf
v = 0 # скорость
mu = 0.99 # коэффициент трения (моментум) [0.5, 0.9, 0.95, 0.99]
learning_rate = 0.1
tol = 1e-7
max_iter = 1e7

for i in range(int(max_iter)):    
    grad = gradient(Theta, Xd, ytrain, m)
    prev_v = v
    v = rho * v - learning_rate * grad
    Theta = Theta - mu * prev_v + (1 + mu) * v
    loss = cost(Xd, ytrain, Theta, m)
    if cost_prev - loss < tol:
        break
    cost_prev = loss

print('Количество шагов:', i)
print('cost:', cost(Xd, ytrain, Theta, m), 'Theta:', Theta)

Xtest_scaled = SC.transform(Xtest)
Xtest_scaled = np.c_[np.ones(Xtest_scaled.shape[0]), Xtest_scaled]
y_pred = sigmoid(np.matmul(Xtest_scaled, Theta))
print("Доля правильно предсказанных классов:", np.sum(ytest == np.where(y_pred >= 0.5, 1, 0))/len(ytest))

Количество шагов: 506
cost: 0.00010631988593405196 Theta: [ 2.32875895  4.99154728  3.96441557 17.95671876 29.4497952 ]
Доля правильно предсказанных классов: 0.9142857142857143


## Градиентный спуск методом RMSProp:

In [108]:
# Градиентный спуск методом RMSProp
            
Theta = np.array([0, 0, 0, 0, 0])
#Theta = np.random.rand(Xd.shape[1])
cost_prev = np.inf
grad_squared = 0
decay_rate = 0.9 # уровень угасания [0.9, 0.99, 0.999]
learning_rate = 0.1
tol = 1e-7
max_iter = 1e7

for i in range(int(max_iter)):    
    grad = gradient(Theta, Xd, ytrain, m)
    grad_squared  = decay_rate * grad_squared + (1 - decay_rate) * grad * grad
    Theta = Theta - learning_rate * grad / (np.sqrt(grad_squared) + 1e-7)
    loss = cost(Xd, ytrain, Theta, m)
    if cost_prev - loss < tol:
        break
    cost_prev = loss

print('Количество шагов:', i)
print('cost:', cost(Xd, ytrain, Theta, m), 'Theta:', Theta)

Xtest_scaled = SC.transform(Xtest)
Xtest_scaled = np.c_[np.ones(Xtest_scaled.shape[0]), Xtest_scaled]
y_pred = sigmoid(np.matmul(Xtest_scaled, Theta))
print("Доля правильно предсказанных классов:", np.sum(ytest == np.where(y_pred >= 0.5, 1, 0))/len(ytest))

Количество шагов: 499
cost: 4.954668021783887e-06 Theta: [ 3.61002725  8.65258334  8.5650289  24.63253094 37.93860235]
Доля правильно предсказанных классов: 0.9142857142857143
